In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import plotly.express as px

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from fuzzywuzzy import fuzz

F:\Projects_Python\Task2\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from gensim.parsing.preprocessing import remove_stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import webcolors

In [4]:
from sklearn.cluster import MiniBatchKMeans

In [5]:
import os

In [6]:
Data_Path = os.path.join(os.getcwd(),'Data')

In [7]:
df = pd.read_csv(os.path.join(Data_Path,'Amazon Search Terms_Search Terms_US.csv'))

In [8]:
df

,Department,Search Term,Search Frequency Rank,#1 Clicked ASIN,#1 Product Title,#1 Click Share,#1 Conversion Share,#2 Clicked ASIN,#2 Product Title,#2 Click Share,#2 Conversion Share,#3 Clicked ASIN,#3 Product Title,#3 Click Share,#3 Conversion Share
0,Amazon.com,pop it,1,B08SLZ712T,"2 Packs Pop Fidget Sensory Toys, Autism Specia...",4.45%,5.21%,B08TVWR4C6,"ASONA Push Bubble Fidget Toy, Bubble Game with...",4.41%,5.22%,B094HBFB18,"Big Size Push Pop Bubble Sensory Fidget Toys,1...",4.15%,2.94%
1,Amazon.com,iphone 11 case,2,B0976787BR,"IMBZBK Shockproof Designed for iPhone 11 Case,...",15.22%,3.80%,B07WQX6V6G,Miracase Liquid Silicone Case Compatible with ...,13.81%,5.59%,B07W4FMQ5Y,"Mkeke Compatible with iPhone 11 Case, Clear Sh...",11.40%,10.69%
2,Amazon.com,bluetooth headphones,3,B095P7K3PV,"Wireless Earbuds, TWS Bluetooth Headphones, St...",11.51%,0.16%,B01HNMTCE2,iJoy Matte Finish Premium Rechargeable Wireles...,8.33%,11.09%,B07RGZ5NKS,TOZO T6 True Wireless Earbuds Bluetooth Headph...,6.57%,9.22%
3,Amazon.com,iphone 12 pro max case,4,B08RNHDV2T,Elando Crystal Clear Case Compatible with iPho...,9.66%,14.46%,B08DTLFHQF,"Mkeke Compatible with iPhone 12 Pro Max Case, ...",6.95%,9.71%,B08DY7M44F,OtterBox Commuter Series Case for iPhone 12 Pr...,6.05%,5.57%
4,Amazon.com,iphone,5,B0775MV9K2,"(Renewed) Apple iPhone 8, US Version, 64GB, Sp...",7.95%,10.92%,B07ZPKN6YR,"(Refurbished) Apple iPhone 11, US Version, 64G...",7.94%,7.42%,B01N4R20RS,"(Refurbished) Apple iPhone 7, 32GB, Black - Fu...",6.80%,13.09%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,Amazon.com,foot massager for plantar fasciitis and neurop...,"999,996",B00SMJIYNG,TheraFlow Dual Foot Massager Roller (Large). R...,29.55%,40.00%,B06WGP2CVL,"Nekteck Foot Massager with Heat, Shiatsu Heate...",29.55%,20.00%,B07F2H1NQR,"RENPHO Foot Massager Machine with Heat, Shiats...",13.64%,10.00%
999996,Amazon.com,free fire 2020,"999,997",B07ZDD1XQM,Squad Fire Survival Shooting Game,58.43%,60.00%,B078N7YVNY,"Knives Out-No rules, just fight!",30.34%,13.33%,B087XCCMPL,FPS Special Encounter Shooting: New Game 2020,8.99%,26.67%
999997,Amazon.com,boot spray deodorizer disinfectant,"999,998",B01N1375XL,"FunkAway Aerospray, 13.5 oz | Extreme Odor Eli...",31.33%,39.29%,B013TSRYUS,Natural Shoe Deodorizer Spray & Foot Odor Elim...,24.10%,21.43%,B01LMMW94Y,10-Seconds Deodorant & Disinfectant (Pack of 2...,12.05%,17.86%
999998,Amazon.com,fenix 5s plus,"999,999",B07D9FNN9W,"Garmin fenix 5s Plus, Smaller-Sized Multisport...",72.06%,90.00%,B08MR1RNXF,Garmin Fenix 5S Plus Sapphire Edition Multi-Sp...,13.97%,10.00%,B07PLNRFRD,"Garmin Fenix 5s Plus, Smaller-Sized Multisport...",9.56%,0.00%


In [9]:
ST = df['Search Term'].sample(
    n=10000,
    random_state=1
)
ST = pd.Series(ST.unique())

In [10]:
ST2 = [remove_stopwords(x)\
        .translate(str.maketrans('','',string.punctuation))\
        .translate(str.maketrans('','',string.digits))\
        for x in ST]

In [11]:
def stemSentence(sentence):
    porter = PorterStemmer()
    token_words = word_tokenize(sentence)
    stem_sentence = [porter.stem(word) for word in token_words]
    return ' '.join(stem_sentence)
ST2 = pd.Series([stemSentence(str(x)) for x in ST2])

In [12]:
colors = list(webcolors.CSS3_NAMES_TO_HEX)
colors = [stemSentence(x) for x in colors if x not in ('bisque','blanchedalmond','chocolate','honeydew','lime',
                                         'olive','orange','plum','salmon','tomato','wheat')]
ST2 = [' '.join([x for x in string.split() if x not in colors]) for string in ST2]

In [13]:
vectorizer_wtf = TfidfVectorizer(analyzer='word')
X_wtf = vectorizer_wtf.fit_transform(ST2)

In [14]:
result = pd.concat([ST,pd.DataFrame(X_wtf.toarray(),columns=vectorizer_wtf.get_feature_names_out())],axis=1)
result.head()

,0,aa,ab,abalon,abandon,abbey,abc,abshoo,absorb,abstract,...,zoot,zorro,zosi,zpresso,zugu,zycam,zymol,zymox,zzzquil,安博
0,queen bedset,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,norb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ring compatible door locks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,floral glue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,snap squad spinosaurus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
vectorizer_cv = CountVectorizer(analyzer='word')
X_cv = vectorizer_cv.fit_transform(ST2)

In [16]:
result = pd.concat([ST,pd.DataFrame(X_cv.toarray(),columns=vectorizer_cv.get_feature_names())],axis=1)
result.head()

F:\Projects_Python\Task2\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0,aa,ab,abalon,abandon,abbey,abc,abshoo,absorb,abstract,...,zoot,zorro,zosi,zpresso,zugu,zycam,zymol,zymox,zzzquil,安博
0,queen bedset,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,norb,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ring compatible door locks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,floral glue,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,snap squad spinosaurus,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
vectorizer_ntf = TfidfVectorizer(analyzer='word',ngram_range=(1,2))
X_ntf = vectorizer_ntf.fit_transform(ST2)

In [18]:
result = pd.concat([ST,pd.DataFrame(X_ntf.toarray(),columns=vectorizer_ntf.get_feature_names_out())],axis=1)
result.head()

,0,aa,aa pack,ab,ab board,ab machin,abalon,abalon jewelri,abandon,abbey,...,zugu,zycam,zymol,zymol car,zymox,zymox plu,zzzquil,zzzquil melatonin,zzzquil night,安博
0,queen bedset,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,norb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ring compatible door locks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,floral glue,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,snap squad spinosaurus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1

In [19]:
kmeans = MiniBatchKMeans(
    n_clusters=200,
    reassignment_ratio = 0.01,
    batch_size = 4096,
)
kmeans.fit(X_cv)

MiniBatchKMeans(batch_size=4096, n_clusters=200)

In [20]:
result = pd.concat([ST,pd.DataFrame(X_cv.toarray(),columns=vectorizer_cv.get_feature_names_out())],axis=1)
result['cluster'] = kmeans.predict(X_cv)

In [21]:
result[[0,'cluster']]

,0,cluster
0,queen bedset,1
1,norb,35
2,ring compatible door locks,84
3,floral glue,124
4,snap squad spinosaurus,156
...,...,...
9995,iphone 11 arm bands for exercise,153
9996,luxury bath rugs,55
9997,xlr to xlr,193
9998,14 mm male piece banger,7


In [22]:
for x in range(len(list(result['cluster']))):
    print(result['cluster'][x],result[0][x])

1 queen bedset
35 norb
84 ring compatible door locks
124 floral glue
156 snap squad spinosaurus
141 standard pillowcases
174 extra long bandaids
146 yellow sheet set
37 beaded jump rope adult fitness
141 16gb ddr4 ram
92 petsafe wireless fence collar
7 omron blood pressure monitor wrist cuff
79 custom napkins
32 vasagle furniture
7 joker blu ray
166 revolution smart toaster
7 hewlett packard laptop
105 the last stand of the tin can sailors
170 portable charger power bank
81 whitey bulger
39 manimal wrist wraps
167 self watering planters and pots
147 rhino sewer hose for rv camper
141 healthy choice frozen meals
139 baby proof outlet covers
5 choco musk
107 hotel style bedding
21 hulu markers
104 bowie shirt
176 mini dresses
105 dog car divider
189 lunch box toddler
92 hanging nail polish rack
147 shower corner bench
84 rustic ring bearer box
188 breathable apple watch band
102 summer comforter
8 nba jerseys for men
8 testicuzzi for men testicle jacuzzi
70 stage 2 baby food
153 gym band

117 blue led light bulbs
79 minions soundtrack
21 badger foot balm
32 bathroom faucets black
12 spongebob diamond painting
58 dovo straight razor
84 titanium belly button rings
124 kindle oasis cellular
104 optical illusion shirt
92 door jammer
5 tasty bite
127 monturas de lentes
148 mini pop it pineapple
91 twin down alternative comforter
127 crocks assecories
13 mom bag for hospital
54 hat womens
63 hamburger hill
193 false teeth for missing teeth
43 pasamontañas para el sol y calor
112 beverage cooler
8 onesies for men
63 river floaties
124 removable glue
8 mens bucket hat
105 twilight books kindle
152 the five dysfunctions of a team
178 disney luggage for adults
193 orfeld vacuum
170 wireless charger portable
195 fly masks for horses with ears
105 massage chair for car
166 65 4k tv
11 dr d
79 cereal dispenser
79 garlic zoom
21 crystal katana
68 white hooks for hanging
183 gaming chair with cup holder
54 smarty pants womens vitamins
32 digital bathroom scales most accurate prime
58 

140 mini cast iron skillet
152 cute shelves
112 gluten free ladyfingers
35 anderson hitch
107 flower bedding
79 tiny magnets
193 vigoro
92 fast wireless charger
68 hammer of the gods
79 polar ignite 2
8 business planner for men
179 bemer therapy mat
1 box spring queen
7 landscaping clothing
105 laptop car mount
92 wireless cam
114 cute school outfits
95 753-08171
141 g.skill tridentz rgb series 16gb
7 written in my own hearts blood
174 can opener oxo
54 nike flex women
131 wicker fruit basket
20 speaker stand desk
103 biker shorts for women high waist plus size
79 infantino blocks
144 usb c chargers
21 kingdom hearts poster
95 straighter
124 fat burning gel
166 wood comb
196 plus size shirt dress for women
147 small ceramic pots for plants
144 anniversary card for her
21 world of warcraft figure
105 graco convertible car seat 4 in 1
115 foot massager machine
66 button down swimsuit cover up
147 rv power surge protector 50 amp
127 battle ground jim butcher
152 fanny pack holster
90 ipho

20 sriracha powder
77 brushless motor and esc 1/10 scale
86 forehead light
135 wood patio table
77 sega
63 lowrance fishfinder
54 cute one piece swimsuit for women
121 rog strix g15 advantage edition
147 cocomelon 1st birthday party supplies
18 teen girl backpack with lunch box
74 stainless steel wire brush
95 bosley
105 the only astrology book you'll ever need
20 groomsmen gifts for wedding
7 portrait of the artist as a young man
105 dvd player car
95 mun
168 diapers size 2 pampers swaddlers
5 hange nendoroid
105 robot air freshener
127 sole f63 treadmill
124 the paris apartment by kelly bowen
156 happy trails
147 bluetooth sports earbuds
105 charmin forever roll starter kit
127 endangered species chocolate
72 honeywell portable air conditioner window kit
105 osmo little genius starter kit
105 book caddy
9 cheap baby girl clothes under 5 dollars
179 cellular shades cordless
162 coil charging cable
156 smithsonian channel
86 tailgate light
191 correa para apple watch series 6
105 glade

84 baby links rings
47 arcade stick ps4
63 finger scooter
135 standing table
95 omfg
155 cigarette machine
105 tempera paint
147 flamingo party decor
188 lv apple watch bands 44mm
86 astro light
135 black outdoor coffee table
57 snow shoes
105 seasoning kit
35 bumbells
51 tension headache
51 tounge vibrator
166 smart tv 50
152 swedish meatballs
58 divination tools
105 personalized dog collar
112 tommy bahama hat
90 11 pro max iphone case
10 plastic forks and knives
147 16 birthday decorations
166 weiman jewelry cleaner
68 sleeping earbuds
141 monopoly deal
123 toddler sheets for toddler bed girl
105 ear cleaner for dogs
142 pencil roll up case
7 office music
58 ice cream for dogs
43 botas de trabajo para hombre wolverine
8 copper fit ankle brace for men
127 thai green curry paste
54 padded bralettes for women
7 glitter sticker paper
170 portable air conditioner cover
153 face exfoliant
7 mountain man romance
77 melissa and doug doll
89 white beaded necklace
75 kitchen carpet
8 mens bro

...

51 noctua nf-a14
187 blue light filter
105 combination wrench sets
162 svideo cable
105 car rim paint
79 sepillo planchador de pelo
131 baskets for blankets
43 pigmentos para uñas
178 zep cleaning products
79 56 degree wedge right hand
150 mens heavyweight t shirts
58 car window cleaner tool with handle
13 moon bag chair
51 e.t.
54 70s bell bottoms for women
127 coppertone glow with shimmer
180 glass birds
153 running band
21 carryon luggage with spinner wheels 22 inch
63 fox farm dirty dozen
124 just a regular mom trying not to raise liberals
105 macbook air 2019
95 pinata 5
39 soda stream aqua fizz
89 igirl necklace
32 mini amplifier
58 unicorn mug
92 baking cooling rack
105 excalibur food dehydrator
141 apple imac 2021
43 house of marley headphones
7 magnetic notebook paper
70 modern baby crib
153 goop sunscreen face
77 playmobil hospital
105 level 2 charger nema 14-50
180 glass dessert cups
32 parasound amplifier
105 cat pellets litter
81 highlighters erasable
57 pencil shoes
21 op

68 vital wheat gluten
21 food signs
54 active sets for women
148 pop it cupcake
147 shower dildo
32 farmhouse bathroom faucet
18 sun hat toddler girls
142 cosmetic case
144 usb dvd burner
8 bamboo underwear men
32 window suncatcher
17 snuggle me organic lounger cover
79 softsoap hand soap
86 sunflower string lights
5 jk shocks
162 pet shaver
127 refresh mega 3
7 to make you feel my love
199 super charger for samsung
95 red green show
79 soap making
18 the baddest girl on the planet
21 wish you were here poster
166 fish emulsion fertilizer for vegetables
7 emt belt
39 andy ngo unmasked
166 tv arm mount swing arm
172 sink legs
174 captain beefheart
87 macbook pro adapter hdmi
153 live tadpoles
10 security camera for car
21 lol big surprise
193 pink underwear
105 book puzzle
105 kenwood single din car stereo
105 clay cat litter
148 celery stick
43 reloj curren hombre
141 creperas electricas
178 retainer brite cleaning tablets
39 phone holder for arm running
153 rubber circles for gym
112 

105 hangyuan fidget sensory toy set
58 green lawn paint
7 gaming headset for xbox one
156 de ruijter
86 15 watt night light bulbs
141 takin
70 its all over now baby blue
105 from cradle to stage book
79 oculus rift cv1
7 gaming mouse pad white
48 large room ceiling fan
193 robot vacuum deebot
127 battle for the soul
105 watermelon decor
39 peloton phone holder
193 black tail magazine
124 rhinestone gel
81 beurer bm55
63 infant bouncers & rockers
118 mystery pallets amazon returns
87 ipad pro 12.9 screen replacement
86 rv tail light license plate bracket
7 bosch season 4
143 anker phone chargers iphone
174 mop microfiber
58 mouthwash for dry mouth
156 pokemon napkins
154 marshmallows large smores
105 flower wall stickers
13 tea bag
95 piccadilly
193 snape
144 256 gb flash thumb drive
92 the sandlot dvd
147 instant pot ultra
79 metal flower
127 pi7 bowers wilkins
21 opal crystal
131 bike basket for back of bike
28 overnight lip mask
166 gopro mouth mount
127 adam serwer
174 nsk dental ha

21 apple pencil 2 tip
20 hawaiian spirulina powder
13 kids duffel bags
21 tyler perry house of payne
7 amythestyst bracelet
68 conditioner
70 puracy baby stain remover
105 auto feeder dog
156 roomba 981
147 fairy garden plants
141 alopecia
79 soy creamer
180 plastic cups with lid
21 propane torch head
96 snack packs for kids
144 baking stone for oven
86 friday night lights season 1
123 bed storage
54 bling keychains for women
43 cascos para bicicletas
92 3 doors down cd
8 jeans levis for men
7 snow gloves
124 pva glue
51 angle drill
77 rain gauge replacement tube glass
123 disc o bunk beds
39 prenatal probiotics
21 pencil grip
68 suavecito matte pomade
39 mystery jigsaw puzzles
43 ropa para bebe
79 drink holders for boats
105 365 days book
86 light room
94 white and gold curtains
17 clothing organizer
71 short extension cord 6 inch
57 soccer shoes for boys
81 couch l shape
105 dog leash for running
32 count dooku lightsaber
51 lifesaver lollipops
21 small pottery wheel
21 nagatoro figu

81 zebra zd410
9 hello baby book
7 adidas golf pants
24 running socks for women
113 three dog bakery dog treats cookies
13 army bag
154 large collapsible bowl
105 braid hair
162 my travel tray
9 baby bottle accessories
77 shark vacmop replacement pads
164 toddler backpacks
172 folders with pockets and prongs
189 tape for boxes
134 white classic luxury bath towels
19 nintendo switch game bundle
9 bath tub baby
105 bling cat collar
7 ufc gloves
81 keto cheetos
105 small placemats
7 144hz 1080p monitor
18 snow white costume for girls
78 pill organizer with case
105 sun shade umbrella for car windshield
123 futtonsleeper sofa bed
166 fireplace glass cleaner
178 bushnell binoculars for adults
105 clock for wall
189 craftsman rolling tool box
66 traeger grill covers
144 usb c angle
123 rubber bed sheet
7 trevor henderson plush
134 towel warmer
51 eufy solo
58 tide pods sport
170 portable air conditioner battery operated
105 dremel set
54 office decor for women desk
141 av hdmi converter
105 

## 2

In [23]:
kmeans = MiniBatchKMeans(
    n_clusters=200,
    reassignment_ratio = 0.01,
    batch_size = 4096,
)
kmeans.fit(X_wtf)

MiniBatchKMeans(batch_size=4096, n_clusters=200)

In [24]:
result = pd.concat([ST,pd.DataFrame(X_wtf.toarray(),columns=vectorizer_wtf.get_feature_names_out())],axis=1)
result['cluster'] = kmeans.predict(X_wtf)

In [25]:
result[[0,'cluster']]

,0,cluster
0,queen bedset,3
1,norb,3
2,ring compatible door locks,3
3,floral glue,3
4,snap squad spinosaurus,3
...,...,...
9995,iphone 11 arm bands for exercise,3
9996,luxury bath rugs,3
9997,xlr to xlr,3
9998,14 mm male piece banger,3


In [26]:
for x in range(len(list(result['cluster']))):
    print(result['cluster'][x],result[0][x])

3 queen bedset
3 norb
3 ring compatible door locks
3 floral glue
3 snap squad spinosaurus
3 standard pillowcases
3 extra long bandaids
3 yellow sheet set
3 beaded jump rope adult fitness
3 16gb ddr4 ram
3 petsafe wireless fence collar
3 omron blood pressure monitor wrist cuff
3 custom napkins
3 vasagle furniture
3 joker blu ray
3 revolution smart toaster
3 hewlett packard laptop
3 the last stand of the tin can sailors
3 portable charger power bank
3 whitey bulger
3 manimal wrist wraps
3 self watering planters and pots
3 rhino sewer hose for rv camper
3 healthy choice frozen meals
3 baby proof outlet covers
3 choco musk
3 hotel style bedding
3 hulu markers
3 bowie shirt
3 mini dresses
3 dog car divider
3 lunch box toddler
3 hanging nail polish rack
3 shower corner bench
3 rustic ring bearer box
3 breathable apple watch band
3 summer comforter
3 nba jerseys for men
3 testicuzzi for men testicle jacuzzi
3 stage 2 baby food
3 gym bands
3 copper vitamin
3 disposable containers with lids
3 b

3 milestone herbicide
3 pinata stick and blindfold
3 small black crossbody purse
3 purple dragon
3 shower light waterproof
3 odoland camping cookware
3 soundproof curtain
3 kayleigh mcenany book
3 custom logo stickers
3 group games for adults
3 easy to clean fan
3 extra fine point dry erase markers
3 chi for dogs
3 grout bag
3 acne face wash benzoyl peroxide
18 indie pants
3 the legend of zorro
3 musical christmas lights
3 ace wrist support
3 transformers titan class
3 levis 517
3 pipe shelf
3 mommys bliss constipation ease baby
3 kong toys for puppies
3 flask for liquor for women
3 peach slices deep blemish microdarts
3 dinosaur pop it fidget toy
3 only natural pet
7 galaxy s21 ultra case with s pen holder
3 white folders
3 jogging shorts for women
165 crystal deodorant stone
3 telescopic poles
3 cup turners for tumblers starter kit
3 colored pencils erasable
3 pouf chairs
3 mint tea
3 tachometer
3 fuel line clamps
3 lunch organizer
3 27x9x14 utv tires
3 pendant light cover
3 display 

3 lettuce dryer
3 baseboard heater thermostat
186 32 inch tv stand
3 fidelio x2hr
3 dashboard lights
3 dolphin float
3 travel size heat protectant spray
3 rook piercing
3 baby bling
3 muscle stimulator for pain relief
3 1 oz hand sanitizer bottles
3 red dead redemption 2 ps4
3 blue curtain lights
133 airpods pro case red
27 school for good and evil
3 12 pro max case otterbox
3 42 inch desk
3 light blue perfume
3 makerbot filament
3 woodburning tips
3 birkenstock arizona sandals women
3 privacy window film bathroom
3 cinderella costume for women
3 just my size panties for women plus size
3 queen bed sheets set
3 starbucks color changing cups
3 una corte de rosas y espinas
3 kill me three times
3 bullet journal stencil
3 ravenclaw robe adult
3 moto one 5g ace case
189 mini popcorn maker
3 black cake drum
3 laundry detergent cup holder & drip tray catcher
3 10 second shoe deodorizer and disinfectant
3 msm powder for hair growth organic
3 countdown to zero day
3 mulberry paper
3 light blue

3 silk wrap for nails
3 fire stick 2nd generation
3 rice and beans dispenser
3 crease shields
3 office chair arm replacement
3 tech armor ipad air 4 screen protector
3 make your bed book
3 soccer chair
3 sandboxes
3 outdoor hanging fan
3 two person sleeping bags for adults
3 tooth brushing holder
3 dc funko pop
3 extra large hammock
3 crew hair products for men
3 swimming tubes
3 collapsible shovel
3 rotating outlet
3 dallas stars jersey
3 feather lily
3 long folding table
3 folding brackets
3 evan moor grade 2
3 cica tape
3 dnd beyond
3 insect repellent lotion
3 columpios para patio para niños
3 corkys shoes women
3 eyeshadow for green eyes
3 samsung galaxy s10 plus tempered glass
3 adidas boost
3 2016 silverado accessories
3 womens backpack cooler
3 dog swimsuit
3 plastic pallet
3 printed jeans
3 oneplus 7t screen protector
3 charlotte hornets
3 bulls shorts
3 apple wach
3 multivitamin for men over 40
3 beats headphones charger
3 boil in bag rice
3 tony horton power life protein powd

18 mens pants big and tall
3 soy candles
3 blue caliper paint
3 yeti lid replacement
3 super mario party decorations
3 liquid molly 5 40 full synthetic
3 black skin care
3 oven mit
3 human fall flat
3 lighting gels
3 birthday decorations for kids
3 wax stamp
3 frying pan lid
3 business ledger
3 nose spray decongestant
3 dutch movie
3 blooming bath flower
3 pull tabs tickets
3 sunscreen tinted
3 zelda diamond painting
3 funko wanda
3 nail tech organizers and storage
3 64 oz insulated water bottle
3 high heel flip flops for women
3 hellhouse llc
3 magnetic usb c charging cable
3 funko pop alice in wonderland
3 redeye
3 running caps for women
3 11x14 canvas frame
3 inflatable pfd coast guard approved
3 110v to 12v converter
3 purple vibrator
3 guy harvey fishing shirts for men
3 corkcicle coffee mug
3 moroccan oil color depositing mask
3 fender medium guitar picks
3 pentel energel pens
3 pill box small
3 trading for dummies
3 add on
3 clay beads for jewelry making preppy
3 sharpie clear v

...

3 aero precision scope mount 30mm
3 cerave facial moisturizing lotion am spf 30
3 sigma 16mm 1.4 canon ef-m
3 lip plumper gloss
3 canon dslr camera
3 traxxas slash shocks
3 blues brothers movie
3 electronic face mask
96 wedding gift
3 penguin backpack
3 turkey bacon
3 eco flow generator
3 martha beck
3 usb computer speaker
3 lavender tablecloth
3 hair darkening shampoo bar
3 compact vacuum cleaner
3 animals coloring book
3 vintage trucker hat
3 a beautiful composition of broken
63 toy keyboard
3 hand strengthening equipment for therapy
3 flower girl shoes for wedding
3 the kings of summer
198 modern shelf
3 among us backdrop
3 samsung galaxy s21 ultra charger
3 tokina 11-16mm f 2.8 nikon
3 sneakers women
3 my lighthouse
3 matrix brass off shampoo
3 arizona tshirt
3 bermudas para hombres
3 silcock water key
3 3949247v
3 my future
3 face tanners best sellers
3 windynation
3 adult electric motorcycle
3 pennant banner
3 unique planter
3 stick chair
3 4 outlet surge protector
3 man cologne


143 otofly iphone 11 pro case
3 trimmer line
3 rogue lifting belt
3 outback pain relief
3 green tea stick
3 brush blow dryer
3 cute bottle opener keychain
3 mini makeup bag
3 blue zones book
3 heart of man
3 link cable
149 the center cannot hold
3 switch lite
3 cow print purse
3 moms on call 0-6 months
3 reishi mushrooms
3 smart home thermostat
3 fake nails coffin
3 flannel pajamas women
3 halloween onesie adult
8 ab machine
3 knee pads for babies
3 irish gifts for women
3 55 gal steel drum
3 ish
3 hu grain free crackers
3 catalyst iphone 11 case
3 paint brushes for kids
3 sloth gifts for women
3 catchers helmet
3 maquillaje al por mayor
3 desk for 2 people
3 fried chicken batter mix
3 britta filter
3 camera for monitor
3 dolphin speaker
3 white photo album
3 cowboy bebop stickers
3 switching power supply 100-240v 50 60hz
3 play money
3 verizon remote control fios
3 club dresses for curvy women
3 semi formal dress
3 half gallon stainless steel water bottle
3 centrum for men
3 cute dino

3 leopard print masks
3 hi pro pac extremely damaged hair intense protein treatment
3 universal rocks
3 magic the gathering land cards
3 2022-2023 monthly planner
186 65 inch smart tv 4k samsung
3 aluminum screen
3 mcculloch
3 cw
3 aveeno kids body wash
3 devil
3 boho pillow cases
3 tarot card deck
3 365 multivitamin
3 jeep soft top straps
3 celery juice book
3 small buffer polisher
3 xs glass screen protector
3 halloween baby blanket
3 airpod silicone ear tips
3 maid costume
3 optix 55
3 ii inin womens deep v neck casual dress backless
3 allegra k dresses for women
3 sugar free black licorice
3 bag holder for car
3 skybrite star projector
3 trump 4th of july shirt
3 loungfly
3 kobe socks
1 dog splash pool
3 grill grate brush
3 microsoft surface laptop 3 case
3 austin peanut butter crackers
3 clarks dress shoes for women
3 happ
3 vans gum sole
3 geometry set for high school
3 karen carpenter
3 jurassic world scorpius rex
3 steering column lock
3 mole and vole repellent
3 camo board sho

3 floating pool chairs for adults
3 magic mount magnetic phone car mount
3 hepagard for fatty liver
3 hicocu 25mm mink lashes
3 stella got her groove back
3 epoxy measuring cups
3 ear plugs for sleeping noise cancelling
3 organizador de joyas
3 revitive medic
3 zantrex black
3 display rack
3 cream colored tops for women
3 rv covers
3 noise cancelling gaming headset
3 natures bounty b12
3 oculus quest 2 vr cover
3 distressed jean shorts
3 rechargeable closet light
3 drain cleaner for septic tanks
3 sioro pajamas for women
3 crop top
3 guinea pig shirt
3 pink chalk paint
3 womens black denim shorts
3 packing foam roll
3 adidas forum mid
3 long mirrors for wall
3 fds
3 6inch cake round pan
3 gavita pro 1700e led
3 womens western dress
3 clips for outdoor string lights
3 office chair with arms
43 sofa bed wood
3 laugh
3 pregnenolone 30mg
3 tidal 55 aquarium filter
3 cast iron holder handle
3 skechers beach casuals
3 zenitsu shoes
3 lumipets
3 shadow box 16x20
68 women scrub set
3 farmhouse

3 really big pop it
3 snakcs
3 juguetes para niñas 4 a 5 años
3 torani salted caramel syrup
3 paris bathroom accessories
3 emerald cut ring
3 school ruler
3 harry potter minalima
3 shark jewelry
3 anker nebula
3 long sleeve white tshirt men
3 perry the platypus onesie
3 canvas for painting
3 4hunnid
3 shortwave uv light for minerals
3 cow print lanyard
3 knife folding
3 galaxy note 10 case with screen protector
3 mask for face skin care
3 protection bracelets for women
3 mitsuri kanroji cosplay
3 250 density human hair wigs
3 celebrity pink jean shorts
3 no se aceptan devoluciones
3 qhohq 3 pack screen protector for iphone 12 pro
3 silicone mold
3 indoor planters
3 gel markers
3 hair dye clips
3 17 inch monitor
3 matthew walker why we sleep
3 t shirt maker
57 andre the giant
3 ge mwf refrigerator water filter
3 beach dresses
100 natural wood coffee table
3 hawaiian shaved ice
3 invincible compendium 3
3 epic elements
3 flaxseed oil 1000mg softgels
3 columbia swim trunks men
3 introvert

## 3

In [27]:
kmeans = MiniBatchKMeans(
    n_clusters=200,
    reassignment_ratio = 0.01,
    batch_size = 4096,
)
kmeans.fit(X_ntf)

MiniBatchKMeans(batch_size=4096, n_clusters=200)

In [28]:
result = pd.concat([ST,pd.DataFrame(X_ntf.toarray(),columns=vectorizer_ntf.get_feature_names_out())],axis=1)
result['cluster'] = kmeans.predict(X_ntf)

In [29]:
result[[0,'cluster']]

,0,cluster
0,queen bedset,144
1,norb,1
2,ring compatible door locks,1
3,floral glue,1
4,snap squad spinosaurus,1
...,...,...
9995,iphone 11 arm bands for exercise,1
9996,luxury bath rugs,1
9997,xlr to xlr,1
9998,14 mm male piece banger,1


In [30]:
for x in range(len(list(result['cluster']))):
    print(result['cluster'][x],result[0][x])

144 queen bedset
1 norb
1 ring compatible door locks
1 floral glue
1 snap squad spinosaurus
1 standard pillowcases
1 extra long bandaids
1 yellow sheet set
1 beaded jump rope adult fitness
1 16gb ddr4 ram
1 petsafe wireless fence collar
1 omron blood pressure monitor wrist cuff
1 custom napkins
1 vasagle furniture
1 joker blu ray
1 revolution smart toaster
1 hewlett packard laptop
1 the last stand of the tin can sailors
1 portable charger power bank
1 whitey bulger
157 manimal wrist wraps
1 self watering planters and pots
1 rhino sewer hose for rv camper
1 healthy choice frozen meals
1 baby proof outlet covers
1 choco musk
1 hotel style bedding
1 hulu markers
1 bowie shirt
1 mini dresses
1 dog car divider
1 lunch box toddler
1 hanging nail polish rack
1 shower corner bench
1 rustic ring bearer box
77 breathable apple watch band
1 summer comforter
1 nba jerseys for men
1 testicuzzi for men testicle jacuzzi
1 stage 2 baby food
1 gym bands
1 copper vitamin
1 disposable containers with lid

1 light up bracelet for kids
1 amber 311
1 cobalt drill bit
1 hummingbird ornament
1 girls birthday party supplies
1 women head scarfs and wraps
1 black labor white wealth by claude anderson
1 weeylite
1 little space ddlg accessories
1 mini remote control car
1 nintendo switch microphone
1 oversized round chair
1 lol toy
1 beach shorts
1 bebe reborn niño
1 razer viper mini wireless
1 moana costume for girls
1 reaistance bands
1 iphone 12 pro max tripod mount
1 survivor greatest hits
1 women capri leggings
1 bed buddy microwave heat pack original
1 men's wrestling shoes
1 ducky 1 2 mini
1 crutches for kids
1 floral picks
1 the machine
1 maracuja oil
1 hgtv tv shows prime free
1 diamond sharpening stone
1 pin press
1 halloween rae dunn
1 coq10 200mg softgels
1 100 ft measuring tape
1 swim shorts women
1 kaws
1 rolling cart
1 55 inch samsung tv
1 boy sandals size 1
1 gummy hair vitamin
1 vermont curry
1 candyman 2020
1 food jibbitz
1 eliptica
1 modern witch tarot deck
1 safety bars for sh

1 dc poster
1 ll cool j greatest hits
1 feminine mystique
1 govee waterproof led strip lights
1 baby proofing
1 leave in conditioner curly hair
1 pool rules signs outdoor funny
1 star trek puzzle
1 palm sander cordless
1 sexy mini dress for women
1 free laundry detergent
1 gps tracking watch for kids
1 8 foot round pool cover
1 smart life led strip lights
1 ph reader
49 sweven perfume men oakcha
1 sheer cover
1 wall mirror art
1 apple pencil 2 accessories
1 samsung powerbot
1 teva womens flip flops
1 nintendo game system
1 fluffy grey rug
1 kids forks
1 extremerate
1 playmobil princess
1 bombtech
1 luvs size 1
1 kitchenaid grinder
1 deviled egg plate
1 greenworks dethatcher
1 apple watch series 3 bands 42mm
1 gillete deodorant liquid for men
1 a80
1 automotive replacement fuses
1 solar panel wiring
1 bird feeder that throws squirrels off
1 craftsman mower deck parts
1 echo show 8 stand
1 travel stickers for scrapbooking
1 range hood insert 30 inch
1 zodiac sign charms
1 timberland sand

1 pug stuff
1 garden covers for raised beds
128 cloud crocs
1 riverdale stuff
1 toddler potty training underwear boys
1 cult documentaries
1 lifevac anti choking device baby
1 chux pads
1 quilt wall hangers
1 toast
1 deep pocket sheet
1 iphone 11 promax privacy screen protector
1 rooster lamp
1 hp officejet 5200 ink cartridge
1 oferta
1 cat repellent outdoor
1 bartholin cyst
1 smart tv remote control universal
1 secret deodorant for women clear gel
1 low sodium crackers
1 pots for cooking
1 split end treatment
1 wool hats for men
1 gas regulator
1 pink surgical masks disposable
1 madison park essentials
1 off white comforter
1 holiday flags
1 trifold lounge chairs for sun tanning
1 desk dividers for students
1 personalized throw blanket
1 laid james
1 the night is still young nicki minaj
1 bathroom set with shower curtain and rug and accessories
1 mini vending machine for snacks
1 mini displayport to hdmi
1 activated charcoal for smells
1 hot pink spray paint
1 cat spoon rest
1 portal 

1 sun flowers home decor clearance
1 3m command strips
1 shrimp wild caught peeled and deveined
1 grown ups dvd 1 and 2
1 ductless ac
1 wave ring
1 friction defense
1 deoderizers-small spaces
1 shark chair
1 can am
1 white button down dress
1 boho prints wall art
1 extra large squishmallow
1 soccer gear for boys
1 knipex needle nose pliers
1 womens saucony shoes
1 tula free to grow baby carrier
1 cricut white iron on vinyl
1 bluetooth hoverboard
1 5w-20 full synthetic oil high mileage
1 olaplex 1 and 2
1 lumeflame light bulb
1 isi whipped cream chargers
1 wings of fire book 5 graphic novel
1 macarons cookies
51 pamela clare
1 laura ashley underwear
1 tetris light
1 3x7 shifters
1 kitten incubator
4 dandelion seeds for planting organic
1 mercruiser spark plugs
1 logitech remote
1 pop rivets assortment
1 paletas de maquillaje
1 owl jewelry for women
1 samsung note 20 charger fast charging
1 samsung 3 in 1 charging station
1 paw patrol chair
1 keto chocolate bars
1 grass pee pads for dogs

...

1 iphone xs slim case
1 counting chips
1 noma
1 harley motorcycle cover
1 2011 f250 headlights
1 vr systems
1 butter molds silicone
1 22-11-8 atv tires
1 dewalt palm sander
1 nama cold press juicer vitality 5800
1 for sale signs for real estate
1 collapsible water bottles 2 pack
1 usa crewneck sweatshirt
1 electric muscle stimulator machine
1 thermacell pads only
1 sunflower cake topper
1 bisexual sticker
1 brown shirt
1 gazebo lights outdoor
1 the time tunnel
1 genexa cold crush
1 phone case se
1 truck bed tent
1 first response ovulation test
1 now house for pets by jonathan adler
1 mercer paring knife
1 lips decor
1 this is my town
1 muscletech clear muscle
1 girls basketball shorts
1 contact paper gold
1 training wheels
1 naruto cat collar
1 long ottoman
1 sign board
1 xtemp tshirt
1 hydration powder packets
1 simple mills almond flour crackers
1 dumbbells rack
1 baby girl toys 6 to 12 months
1 personalized cups
1 north face backpack kids
1 pill organizer 2 times a day
1 indian styl

1 led strip lights warm white
1 hat strap
1 protector de colchon twin
1 wod nation
1 caftans or long dresses
1 ceramic cactus
1 yellow pencil pouch
1 lunch bag ice pack
1 shower liner with magnets
1 kyoetsu
1 zooby kin car baby monitor
1 purity coffee organic mold free
1 best friends book
1 traeger pizza stone
1 corner gas animated
1 straight edge guide
1 glowshift gauges
1 texas rangers
1 disney infinity 2.0 characters
1 pledge multisurface wipes
1 camping stuff for kids
1 roccat headset
1 cayenne pepper bulk
1 cuna para bebe recien nacido
1 toy car set
1 argyle sweater vest women
1 waterproof mattress cover king
1 id holders for badges women
1 pieces of her karin slaughter
1 pillow insert 20x20
1 beauty by bianca
1 dead head props
1 barbie princess and the pauper
1 tall in the saddle
1 lamp for desk
1 pur cosmetics
1 crawlspace door
1 osmo numbers
1 tennis racket 23 inch
1 thin silver hoop earrings
1 card wars
1 authenticator app
1 gun wrap
1 pill box cute
1 office space in prime vid

1 witchcraft supplies kit
1 ductless range hood
1 pocket holster
1 depends for men
1 1 binder with clear cover
1 comforter filler
1 womens halloween shirts
1 fenugreek seeds
1 hunter hunter merch
1 sistema microwave cookware
1 mk shoes for women
1 campus backpack
1 baby bloomers girl
1 xbox series x headset
1 logitech yoke
1 skil screwdriver
1 turface
1 green buddha necklace
1 ihave toothpaste dispenser
1 puckett game
1 murad skin care
1 fluffy wheel cover
1 key remover
1 36x36 table
1 apple pencil 2 tip
1 hawaiian spirulina powder
1 kids duffel bags
1 tyler perry house of payne
1 amythestyst bracelet
1 conditioner
1 puracy baby stain remover
1 auto feeder dog
1 roomba 981
1 fairy garden plants
1 alopecia
1 soy creamer
1 plastic cups with lid
1 propane torch head
1 snack packs for kids
1 baking stone for oven
1 friday night lights season 1
1 bed storage
1 bling keychains for women
1 cascos para bicicletas
1 3 doors down cd
1 jeans levis for men
1 snow gloves
1 pva glue
1 angle drill
1 

1 dark blue hoodie
1 wine siphon
1 dialysis
1 turf shoe
1 tablets on sale prime 10 inch samsung galaxy
1 diamond painting christmas
1 outdoor swing with canopy
1 little house on the prairie complete series
1 zebra zd410
1 hello baby book
1 adidas golf pants
1 running socks for women
1 three dog bakery dog treats cookies
1 army bag
1 large collapsible bowl
1 braid hair
1 my travel tray
1 baby bottle accessories
1 shark vacmop replacement pads
1 toddler backpacks
1 folders with pockets and prongs
1 tape for boxes
1 white classic luxury bath towels
1 nintendo switch game bundle
1 bath tub baby
1 bling cat collar
1 ufc gloves
1 keto cheetos
1 small placemats
1 144hz 1080p monitor
1 snow white costume for girls
1 pill organizer with case
1 sun shade umbrella for car windshield
1 futtonsleeper sofa bed
1 fireplace glass cleaner
1 bushnell binoculars for adults
1 clock for wall
1 craftsman rolling tool box
1 traeger grill covers
1 usb c angle
1 rubber bed sheet
1 trevor henderson plush
1 towe

1 blindfold
1 pop rivets
1 pride flag with pole
1 hpi racing
1 ring with initial
1 adidas cloudfoam women slides
1 groom style wedding
1 packout
1 tickless
1 beach chair lounger
1 tin tub
1 messi shirt
1 personal librarian
1 sony xperia 1 iii pre order
1 love notes for him
1 book light bookmark
1 little tikes climber and slide
1 active romper
1 snap barrettes
1 flower shaped rug
1 pigeon nest
1 phone box
1 nopal cactus capsules
1 spiderman candy
1 imagine dragons wrecked
1 clear septum ring
1 llama party supplies
1 orange boating safety flag
1 pork
1 unicorn gold
1 men's golf shoes
1 homecoming dresses short
1 twilight princess manga
1 number cake molds for baking
1 velvet curtain
1 open water swim buoy
1 12.9 inch ipad pro case
1 hose diverter 2 way
1 maymom flange inserts
1 crystal glasses drinking
1 trailmaker backpack
1 millet spray
1 clear reusable cups with lids and straws
1 weber kettle grill grate
1 fabric labels for quilts
1 pillow top mattress
1 fiestaware
1 grindz grinder cl